In [ ]:
---
layout: post
categories: [Overworld Lessons]
title: Overworld Level Profiles 
description:  Programmer Profiles 
type: issues 
comments: true
permalink: /overworld/EnemyCollision1
---

# Enemy Collisions

Our game is already filled with different types of collision reactions with NPCs. However, in order to make our game even more challenging, we can introduce the idea of having an Enemy. Having an enemy doesn't come without its consequences. In this lesson, you will learn how to create consequences when our player collides with an enemy. 

### Creating an Enemy
In order to make our enemy, we decided to create a new class for it. We made a new .js file titled 'Enemy.js'. Like the other aspects of our game, this allows us to store all the functions for the enemy and create consequences for collisions without making a mess of our code. We extended this class from our character class. There it is:

```js
import Character from './Character.js';
import Player from './Player.js';

class Enemy extends Character {
    constructor(data = null, gameEnv = null) {
        super(data, gameEnv);
        this.playerDestroyed = false; // Tracks if the player has been "killed"
    }

    // Overrides the update method to handle collision detection.
    update() {
        // Start by drawing the object
        this.draw();

        // Check if the enemy collides with the player
        if (!this.playerDestroyed && this.collisionChecks()) {
            this.handleCollisionEvent();
        }
    }

    // Checks if the Enemy collides with the Player.
    // Returns true if a collision is detected, otherwise false.
    collisionChecks() {
        for (const gameObj of this.gameEnv.gameObjects) {
            if (gameObj instanceof Player) {
                this.isCollision(gameObj);
                if (this.collisionData.hit) {
                    return true;
                }
            }
        }
        return false;
    }

    // Handles what happens when the player collides with the enemy.
    handleCollisionEvent() {
        console.log("Player collided with the Enemy. Player is dead.");
        this.playerDestroyed = true; // Mark the player as "dead"
        this.gameEnv.gameControl.restartLevel(); // Restart the level
    }
}

export default Enemy;
;
```
Having the enemy be its own special case file is important because it allows for the enemy to be able to restart the game and have the player die. It being its own class allows it to have special consequences that affect the gameplay.

In this version, we have it so that the player dies and the game automatically resets after collision. There are no animations in this version--that's for the next lesson. 

### Explanation:
This file is very similar to the NPC.js except in this case, it creates a way for something to happen when the player and enemy collide with each other.

For example, in the very beginning of the file, it checks if the player is destroyed (`this.playerDestroyed = false`) and we set it to false so that it can be changed to true later on in the code. 

Then it goes to collisionChecks which simply checks if the enemy is colliding with the player. If the player is collided with the enemy then `this.collisionData.hit` returns true because the player and the enemy are colliding. If its true, a `handleCollisionEvent` occurs, which runs the things that happen when they are collided.

* We used `console.log` to print a message after they collide
* `this.playerDestroyed = false` is changed to `this.playerDestroyed = true`, what we were talking about in the beginning player should be 'dead'
* `this.gameEnv.gameControl.restartLevel()`; this one's more complicated as you had to create a function inside of gameControl that restarts the level. We are simply just recalling it.


Now, in order to make more specific functions for the creeper, we created another class extending from the enemy class. We named it 'Creeper.js.' Here it is:

```js
import Enemy from './Enemy.js';
import GameEnv from './GameEnv.js';
import Player from './Player.js';

class Creeper extends Enemy {
    constructor(data = null, gameEnv = null) {
        super(data, gameEnv);
    }
   

    /**
     * Check for proximity of objects.
     * This method checks if any players are within a certain distance of the creeper.
     * If players are within the specified distance, their names are collected and a response is generated.
     */
    checkProximityToPlayer() {
        //this.velocity.x=10
        // Filter all Player objects from the game environment
        var players = this.gameEnv.gameObjects.filter(obj => obj instanceof Player);
        var creeper = this;
        var names = [];

        var player ;

        if (players.length > 0 && creeper) {
            players.forEach(player => {

                if (player.spriteData && player.spriteData.name == 'mainplayer') {
                    player = player
                    //showCustomAlert('hello')
                    // Calculate the distance between the creeper and the player
                    
                    var deltax= player.position.x - this.position.x
                    var deltay = player.position.y - this.position.y

                    // // The Euclidean distance between two points in a 2D space
                    var distance = Math.sqrt(
                         Math.pow(player.position.x - this.position.x, 2) + Math.pow(player.position.y - this.position.y, 2)
                    );
                    // The distance is less than 10 pixels
                     if (distance > 10) {
                        if(deltax>0){
                            this.velocity.x = this.gameEnv.innerWidth * 0.0005 
                        }
                        else{
                            this.velocity.x = this.gameEnv.innerWidth * -0.0005 
                        }
                        if(deltay>0){
                            this.velocity.y = this.gameEnv.innerHeight * 0.0005
                        }
                        else{
                            this.velocity.y = this.gameEnv.innerHeight * -0.0005 
                        }
                        //this.velocity.x = deltax/200
                        //this.velocity.y = deltay/200
                     }
                    // else {
                    //      this.velocity.x = 0
                    //      this.velocity.y = 0
                    // }

                }
            });
        }


    }

    // Override the jump method for creeper
    jump() {
        // Implement creeper-specific jump logic here
        // console.log("creeper is jumping!");
        //this.y -= 50; // Example: Move the creeper up by 50 pixels
    }

    handleCollisionEvent() {
        //extract player object
        var player = this.gameEnv.gameObjects.find(obj => obj instanceof Player); 
        //collided object is player
        if (player.id = this.collisionData.touchPoints.other.id) {
            
            console.log("Creeper collided with player!");

        // Stop movement
        this.velocity.x = 0;
        this.velocity.y = 0;

        // Explode player object with animation
        this.explode(player.position.x, player.position.y);
        player.destroy();
        this.playerDestroyed = true;

        // Restart level after explosion animation
        setTimeout(() => {
            this.gameEnv.gameControl.restartLevel();
        }, 3000); // Adjust delay based on explosion animation duration
        }
    }

    // Override other methods if needed
}


export default Creeper;
```
